<a href="https://colab.research.google.com/github/rkrissada/google_ml_training/blob/master/5_train_bqml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting babyweight using BigQuery ML

This notebook illustrates:
<ol>
<li> Machine Learning using BigQuery
<li> Jupyter Magic for BigQuery in Cloud Datalab
</ol>

Please see [this notebook](1_explore.ipynb) for more context on this problem and how the features were chosen.

In [0]:
# change these to try this notebook out
PROJECT = 'qwiklabs-gcp-03f3aae32d6f97d5'
REGION = 'asia-southeast1'

In [0]:
import os
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [0]:
%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


## Exploring the Data

Here, we will be taking natality data and training on features to predict the birth weight.

The CDC's Natality data has details on US births from 1969 to 2008 and is available in BigQuery as a public data set. More details: https://bigquery.cloud.google.com/table/publicdata:samples.natality?tab=details

Lets start by looking at the data since 2000 with useful values > 0!

In [0]:
%%bq query
SELECT
    *
FROM
  publicdata.samples.natality
WHERE
  year > 2000
  AND gestation_weeks > 0
  AND mother_age > 0
  AND plurality > 0
  AND weight_pounds > 0
LIMIT 10

source_year,year,month,day,wday,state,is_male,child_race,weight_pounds,plurality,apgar_1min,apgar_5min,mother_residence_state,mother_race,mother_age,gestation_weeks,lmp,mother_married,mother_birth_state,cigarette_use,cigarettes_per_day,alcohol_use,drinks_per_week,weight_gain_pounds,born_alive_alive,born_alive_dead,born_dead,ever_born,father_race,father_age,record_weight
2001,2001,5,,3,FL,True,9,6.686620406459999,1,99,9,FL,2,18,43,07992000,False,Foreign,,,False,,40,0,0,0,1,9,99,1
2001,2001,8,,2,KS,True,9,9.36082764452,1,99,9,KS,1,32,41,10232000,True,KS,,,False,,36,0,0,1,1,1,34,1
2001,2001,9,,2,KY,False,9,8.437090766739999,1,99,9,KY,1,30,39,12992000,True,KY,,,False,,40,0,0,0,1,1,30,1
2002,2002,10,,2,AL,False,9,6.1244416383599996,1,99,10,AL,1,24,40,01072002,True,AL,,,False,,27,0,0,0,1,1,26,1
2002,2002,5,,5,AZ,False,9,7.12534030784,1,99,9,AZ,1,26,41,08152001,False,AZ,,,False,,39,3,0,1,4,1,26,1
2001,2001,10,,3,CA,False,9,6.944561253,1,99,99,CA,1,31,40,01122001,True,IL,,,,,99,0,0,0,1,1,27,1
2001,2001,8,,2,MS,True,9,6.9996768185,1,99,10,MS,1,23,40,11012000,True,MS,,,False,,32,2,0,2,3,1,25,1
2001,2001,2,,6,NY,True,9,9.37405538024,1,99,9,NY,1,34,40,04252000,True,NJ,,,False,,48,1,0,0,2,1,32,1
2001,2001,11,,3,OR,True,9,8.37315671076,1,99,8,OR,1,33,40,02132001,True,CA,,,False,,25,1,0,1,2,1,40,1
2001,2001,7,,2,TX,False,9,7.1870697412,1,99,99,TX,1,23,40,09212000,False,TX,,,False,,24,0,0,0,1,9,99,1


## Define Features

Looking over the data set, there are a few columns of interest that could be leveraged into features for a reasonable prediction of approximate birth weight.

Further, some feature engineering may be accomplished with the BigQuery `CAST` function -- in BQML, all strings are considered categorical features and all numeric types are considered continuous ones.

The hashmonth is added so that we can repeatably split the data without leakage -- we want all babies that share a birthday to be either in training set or in test set and not spread between them (otherwise, there would be information leakage when it comes to triplets, etc.)

In [0]:
%%bq query
SELECT
    weight_pounds, -- this is the label; because it is continuous, we need to use regression
    CAST(is_male AS STRING) AS is_male,
    mother_age,
    CAST(plurality AS STRING) AS plurality,
    gestation_weeks,
    ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE
  year > 2000
  AND gestation_weeks > 0
  AND mother_age > 0
  AND plurality > 0
  AND weight_pounds > 0
LIMIT 10

weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
7.56846945446,true,22,1,46,1403073183891835564
8.8074673669,true,39,1,42,1088037545023002395
8.313631900019999,true,23,1,35,2126480030009879160
8.000575487979999,false,27,1,40,7170969733900686954
6.56316153974,false,29,1,39,3408502330831153141
7.12534030784,false,34,1,40,7108882242435606404
7.43839671988,false,31,1,38,6392072535155213407
7.3524164377,true,30,1,37,8904940584331855459
8.062304921339999,true,16,1,40,2126480030009879160
7.25100379718,true,17,1,39,1525201076796226340


## Train Model

With the relevant columns chosen to accomplish predictions, it is then possible to create (train) the model in BigQuery. First, a dataset will be needed store the model. (if this throws an error in Datalab, simply create the dataset from the BigQuery console).

In [0]:
%%bash
bq --location=ASIA mk -d demo

BigQuery error in mk operation: Cannot contact server. Please try again.
Traceback: Traceback (most recent call last):
File "/tools/google-cloud-sdk/platform/bq/bigquery_client.py", line 889, in
BuildApiClient
response_metadata, discovery_document = http.request(discovery_url)
File
"/tools/google-cloud-sdk/platform/bq/third_party/oauth2client_4_0/transport.py",
line 176, in new_request
redirections, connection_type)
File
"/tools/google-cloud-sdk/platform/bq/third_party/oauth2client_4_0/transport.py",
line 283, in request
connection_type=connection_type)
File "/tools/google-cloud-sdk/platform/bq/third_party/httplib2/__init__.py",
line 1626, in request
(response, content) = self._request(conn, authority, uri, request_uri, method,
body, headers, redirections, cachekey)
File "/tools/google-cloud-sdk/platform/bq/third_party/httplib2/__init__.py",
line 1368, in _request
(response, content) = self._conn_request(conn, request_uri, method, body,
headers)
File "/tools/google-cloud-sdk/platform/b

With the demo dataset ready, it is possible to create a linear regression model to train the model.

This will take approximately **4 minutes** to run and will show **Done** when complete.

In [0]:
%%bq query
CREATE or REPLACE MODEL demo.babyweight_model_asis
OPTIONS
  (model_type='linear_reg', labels=['weight_pounds']) AS
  
WITH natality_data AS (
  SELECT
    weight_pounds,-- this is the label; because it is continuous, we need to use regression
    CAST(is_male AS STRING) AS is_male,
    mother_age,
    CAST(plurality AS STRING) AS plurality,
    gestation_weeks,
    ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
  FROM
    publicdata.samples.natality
  WHERE
    year > 2000
    AND gestation_weeks > 0
    AND mother_age > 0
    AND plurality > 0
    AND weight_pounds > 0
)

SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks
FROM
    natality_data
WHERE
  MOD(hashmonth, 4) < 3  -- select 75% of the data as training


notFound: Not found: Dataset qwiklabs-gcp-03f3aae32d6f97d5:demo was not found in location US

## Training Statistics

During the model training (and after the training), it is possible to see the model's training evaluation statistics.

For each training run, a table named `<model_name>_eval` is created. This table has basic performance statistics for each iteration.

While the new model is training, review the training statistics in the BigQuery UI to see the below model training: https://bigquery.cloud.google.com/

Since these statistics are updated after each iteration of model training, you will see different values for each refresh while the model is training.

The training details may also be viewed after the training completes from this notebook.

In [0]:
%%bq query
SELECT * FROM ML.TRAINING_INFO(MODEL demo.babyweight_model_asis);


notFound: Not found: Dataset qwiklabs-gcp-03f3aae32d6f97d5:demo was not found in location US

Some of these columns are obvious although what do the non-specific ML columns mean (specific to BQML)?

**training_run** - Will be zero for a newly created model. If the model is re-trained using warm_start, this will increment for each re-training.

**iteration** - Number of the associated `training_run`, starting with zero for the first iteration.

**duration_ms** - Indicates how long the iteration took (in ms).

Note: You can also see these stats by refreshing the BigQuery UI window, finding the `<model_name>` table, selecting on it, and then the Training Stats sub-header.

Let's plot the training and evaluation loss to see if the model has an overfit.

In [0]:
import google.datalab.bigquery as bq
df = bq.Query("SELECT * FROM ML.TRAINING_INFO(MODEL demo.babyweight_model_asis)").execute().result().to_dataframe()
# plot both lines in same graph
import matplotlib.pyplot as plt
plt.plot( 'iteration', 'loss', data=df, marker='o', color='orange', linewidth=2)
plt.plot( 'iteration', 'eval_loss', data=df, marker='', color='green', linewidth=2, linestyle='dashed')
plt.xlabel('iteration')
plt.ylabel('loss')
plt.legend();

Exception: notFound: Not found: Dataset qwiklabs-gcp-03f3aae32d6f97d5:demo was not found in location US

As you can see, the training loss and evaluation loss are essentially identical. We do not seem to be overfitting.

## Make a Prediction with BQML using the Model

With a trained model, it is now possible to make a prediction on the values. The only difference from the second query above is the reference to the model. The data has been limited (`LIMIT 100`) to reduce amount of data returned.

When the `ml.predict` function is leveraged, output prediction column name for the model is `predicted_<label_column_name>`.

In [0]:
%%bq query
SELECT
  *
FROM
  ml.PREDICT(MODEL demo.babyweight_model_asis,
      (SELECT
        weight_pounds,
        CAST(is_male AS STRING) AS is_male,
        mother_age,
        CAST(plurality AS STRING) AS plurality,
        gestation_weeks
      FROM
        publicdata.samples.natality
      WHERE
        year > 2000
        AND gestation_weeks > 0
        AND mother_age > 0
        AND plurality > 0
        AND weight_pounds > 0
    ))
LIMIT 100


notFound: Not found: Dataset qwiklabs-gcp-03f3aae32d6f97d5:demo was not found in location US

# More advanced...

In the original example, we were taking into account the idea that if no ultrasound has been performed, some of the features (e.g. is_male) will not be known. Therefore, we augmented the dataset with such masked features and trained a single model to deal with both these scenarios.

In addition, during data exploration, we learned that the data size set for mothers older than 45 was quite sparse, so we will discretize the mother age.

In [0]:
%%bq query
SELECT
    weight_pounds,
    CAST(is_male AS STRING) AS is_male,
    IF(mother_age < 18, 'LOW',
         IF(mother_age > 45, 'HIGH',
            CAST(mother_age AS STRING))) AS mother_age,
    CAST(plurality AS STRING) AS plurality,
    CAST(gestation_weeks AS STRING) AS gestation_weeks,
    ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
  FROM
    publicdata.samples.natality
  WHERE
    year > 2000
    AND gestation_weeks > 0
    AND mother_age > 0
    AND plurality > 0
    AND weight_pounds > 0
LIMIT 25

weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
7.1319541757,true,32,1,28,6691862025345277042
7.3744626639,true,33,1,39,7146494315947640619
8.062304921339999,false,26,1,40,7146494315947640619
7.4846937949,false,24,1,39,3408502330831153141
5.74965579296,false,21,2,36,1088037545023002395
6.1883756943399995,true,22,1,36,5896567601480310696
2.0392759235,false,26,1,28,6691862025345277042
5.18747702486,true,34,2,37,8904940584331855459
7.5618555866,false,34,1,38,2126480030009879160
4.6407306151,false,30,1,39,7108882242435606404


On the same dataset, will also suppose that it is unknown whether the child is male or female (on the same dataset) to simulate that an ultrasound was not been performed.

In [0]:
%%bq query
SELECT
    weight_pounds,
    'Unknown' AS is_male,
    IF(mother_age < 18, 'LOW',
         IF(mother_age > 45, 'HIGH',
            CAST(mother_age AS STRING))) AS mother_age,
    IF(plurality > 1, 'Multiple', 'Single') AS plurality,
    CAST(gestation_weeks AS STRING) AS gestation_weeks,
    ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
  FROM
    publicdata.samples.natality
  WHERE
    year > 2000
    AND gestation_weeks > 0
    AND mother_age > 0
    AND plurality > 0
    AND weight_pounds > 0
LIMIT 25

weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
8.81849048,Unknown,LOW,Single,42,1403073183891835564
8.14167133566,Unknown,29,Single,38,8904940584331855459
5.94807182876,Unknown,38,Single,38,7108882242435606404
8.83833208358,Unknown,27,Single,39,3408502330831153141
9.259415004,Unknown,28,Single,38,1088037545023002395
7.12534030784,Unknown,34,Single,39,7146494315947640619
7.06361087448,Unknown,18,Single,40,1525201076796226340
7.50012615324,Unknown,31,Single,44,1525201076796226340
6.2501051276999995,Unknown,23,Single,41,1525201076796226340
3.99918543268,Unknown,30,Single,32,7146494315947640619


Bringing these two separate data sets together, there is now a dataset for male or female children determined with ultrasound or unknown if without. 

In [0]:
%%bq query
WITH with_ultrasound AS (
  SELECT
    weight_pounds,
    CAST(is_male AS STRING) AS is_male,
    IF(mother_age < 18, 'LOW',
         IF(mother_age > 45, 'HIGH',
            CAST(mother_age AS STRING))) AS mother_age,
    CAST(plurality AS STRING) AS plurality,
    CAST(gestation_weeks AS STRING) AS gestation_weeks,
    ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
  FROM
    publicdata.samples.natality
  WHERE
    year > 2000
    AND gestation_weeks > 0
    AND mother_age > 0
    AND plurality > 0
    AND weight_pounds > 0
),

without_ultrasound AS (
  SELECT
    weight_pounds,
    'Unknown' AS is_male,
    IF(mother_age < 18, 'LOW',
         IF(mother_age > 45, 'HIGH',
            CAST(mother_age AS STRING))) AS mother_age,
    IF(plurality > 1, 'Multiple', 'Single') AS plurality,
    CAST(gestation_weeks AS STRING) AS gestation_weeks,
    ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
  FROM
    publicdata.samples.natality
  WHERE
    year > 2000
    AND gestation_weeks > 0
    AND mother_age > 0
    AND plurality > 0
    AND weight_pounds > 0
),

preprocessed AS (
  SELECT * from with_ultrasound
  UNION ALL
  SELECT * from without_ultrasound
)

SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks
FROM
    preprocessed
WHERE
  MOD(hashmonth, 4) < 3
LIMIT 25

weight_pounds,is_male,mother_age,plurality,gestation_weeks
8.81849048,Unknown,LOW,Single,42
5.94807182876,Unknown,38,Single,38
8.83833208358,Unknown,27,Single,39
7.06361087448,Unknown,18,Single,40
7.50012615324,Unknown,31,Single,44
6.2501051276999995,Unknown,23,Single,41
6.1178277705,Unknown,24,Single,36
9.12493302418,Unknown,26,Single,40
8.062304921339999,Unknown,23,Single,38
8.02703095942,Unknown,28,Single,39


## Create a new model

With a data set which has been feature engineered, it is ready to create model with the `CREATE or REPLACE MODEL` statement

This will take **5-10 minutes** and will show **Done** when complete.

In [0]:
%%bq query
CREATE or REPLACE MODEL demo.babyweight_model_fc
OPTIONS
  (model_type='linear_reg', labels=['weight_pounds']) AS
  
WITH with_ultrasound AS (
  SELECT
    weight_pounds,
    CAST(is_male AS STRING) AS is_male,
    IF(mother_age < 18, 'LOW',
         IF(mother_age > 45, 'HIGH',
            CAST(mother_age AS STRING))) AS mother_age,
    CAST(plurality AS STRING) AS plurality,
    CAST(gestation_weeks AS STRING) AS gestation_weeks,
    ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
  FROM
    publicdata.samples.natality
  WHERE
    year > 2000
    AND gestation_weeks > 0
    AND mother_age > 0
    AND plurality > 0
    AND weight_pounds > 0
),

without_ultrasound AS (
  SELECT
    weight_pounds,
    'Unknown' AS is_male,
    IF(mother_age < 18, 'LOW',
         IF(mother_age > 45, 'HIGH',
            CAST(mother_age AS STRING))) AS mother_age,
    IF(plurality > 1, 'Multiple', 'Single') AS plurality,
    CAST(gestation_weeks AS STRING) AS gestation_weeks,
    ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
  FROM
    publicdata.samples.natality
  WHERE
    year > 2000
    AND gestation_weeks > 0
    AND mother_age > 0
    AND plurality > 0
    AND weight_pounds > 0
),

preprocessed AS (
  SELECT * from with_ultrasound
  UNION ALL
  SELECT * from without_ultrasound
)

SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks
FROM
    preprocessed
WHERE
  MOD(hashmonth, 4) < 3


notFound: Not found: Dataset qwiklabs-gcp-03f3aae32d6f97d5:demo was not found in location US

## Training Statistics

While the new model is training, review the training statistics in the BigQuery UI to see the below model training: https://bigquery.cloud.google.com/

The training details may also be viewed after the training completes from this notebook.

In [0]:
import google.datalab.bigquery as bq
df = bq.Query("SELECT * FROM ML.TRAINING_INFO(MODEL demo.babyweight_model_fc)").execute().result().to_dataframe()
# plot both lines in same graph
import matplotlib.pyplot as plt
plt.plot( 'iteration', 'loss', data=df, marker='o', color='orange', linewidth=2)
plt.plot( 'iteration', 'eval_loss', data=df, marker='', color='green', linewidth=2, linestyle='dashed')
plt.xlabel('iteration')
plt.ylabel('loss')
plt.legend();

Exception: notFound: Not found: Dataset qwiklabs-gcp-03f3aae32d6f97d5:demo was not found in location US

## Make a prediction with the new model



Perhaps it is of interest to make a prediction of the baby's weight given a number of other factors: Male, Mother is 28 years old, Mother will only have one child, and the baby was born after 38 weeks of pregnancy.

To make this prediction, these values will be passed into the SELECT statement.

In [0]:
%%bq query
SELECT
  *
FROM
  ml.PREDICT(MODEL demo.babyweight_model_fc,
      (SELECT
          'True' AS is_male,
          '28' AS mother_age,
          '1' AS plurality,
          '38' AS gestation_weeks
    ))


notFound: Not found: Dataset qwiklabs-gcp-03f3aae32d6f97d5:demo was not found in location US

<br>
<br>
<br>
<br>
Copyright 2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License